In [ ]:
!pip install datasets transformers evaluate optuna
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 req

In [ ]:
from datasets import load_dataset
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import evaluate
from tqdm import tqdm

In [ ]:
# Load the IMDB dataset
imdb = load_dataset("imdb")
test_dataset = imdb['test'].shuffle(seed=42)
print(f"Test dataset size: {len(test_dataset)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Test dataset size: 25000


In [ ]:
# Train the model
print("Starting training...")
train_results = trainer.train()
print(train_results)

# Evaluate on validation set
print("Evaluating on validation set...")
val_results = trainer.evaluate(eval_dataset=tokenized_val)
print(val_results)

# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(eval_dataset=tokenized_test)
print(test_results)

# Save the final model
model_path = "./llama_3.2_imdb_sentiment"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
# Define prompt templates for sentiment analysis
def create_prompt(review):
    return f"Review: {review}\n. Sentiment (positive or negative): "

# Function to get sentiment prediction from GPT-2
def predict_sentiment(review, max_attempts=3):
    # Truncate the review to prevent exceeding the model's context length
    # GPT-2 has a max length of 1024 tokens, but we need to leave room for the prompt and generated response
    max_review_length = 800  # This gives plenty of space for the prompt and generation

    # Tokenize the review to count tokens
    review_tokens = tokenizer.encode(review)
    if len(review_tokens) > max_review_length:
        # Truncate to max_review_length tokens
        truncated_review_tokens = review_tokens[:max_review_length]
        review = tokenizer.decode(truncated_review_tokens)

    prompt = create_prompt(review)

    for attempt in range(max_attempts):
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate text
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=5,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False  # Use greedy decoding for deterministic results
            )

        # Decode the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Extract the sentiment prediction
        prediction = generated_text.split("Sentiment (positive or negative): ")[-1].strip().lower()

        # Check if "positive" or "negative" was generated
        if "positive" in prediction:
            return "positive", attempt + 1
        elif "negative" in prediction:
            return "negative", attempt + 1

    return "unknown", attempt + 1

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")


In [ ]:
# Test GPT-2 on the dataset
predictions = []
ground_truth = []
attempt_counts = []

for item in tqdm(test_dataset, desc="Evaluating GPT-2"):
    review = item["text"]
    true_label = "positive" if item["label"] == 1 else "negative"

    # Get GPT-2's prediction
    prediction, attempts = predict_sentiment(review)

    predictions.append(prediction)
    ground_truth.append(true_label)
    attempt_counts.append(attempts)

    # Print some examples as we go
    if len(predictions) % 1000 == 0:
        print(f"\nReview: {review[:100]}...")
        print(f"True sentiment: {true_label}")
        print(f"Predicted sentiment: {prediction}")
        print(f"Required {attempts} attempt(s)")

# Calculate metrics
valid_predictions = [p for p, g in zip(predictions, ground_truth) if p != "unknown"]
valid_ground_truth = [g for p, g in zip(predictions, ground_truth) if p != "unknown"]

# Report results
total_samples = len(predictions)
valid_samples = len(valid_predictions)
unknown_samples = total_samples - valid_samples

print("\n===== GPT-2 Zero-Shot Sentiment Analysis Results =====")
print(f"Model: {model_name}")
print(f"Total samples: {total_samples}")
print(f"Valid predictions: {valid_samples} ({valid_samples/total_samples*100:.2f}%)")
print(f"Unknown predictions: {unknown_samples} ({unknown_samples/total_samples*100:.2f}%)")

Evaluating GPT-2:   4%|▍         | 1001/25000 [03:00<1:11:21,  5.61it/s]


Review: This is the best version of Gypsy that has been filmed.Bette Midler is simply superb as Mama Rose.Sh...
True sentiment: positive
Predicted sentiment: unknown
Required 3 attempt(s)


Evaluating GPT-2:   8%|▊         | 2001/25000 [05:59<1:11:58,  5.33it/s]


Review: Age of Steel follows up the previous episode, Rise of the Cybermen, which was excellent in some resp...
True sentiment: positive
Predicted sentiment: unknown
Required 3 attempt(s)


Evaluating GPT-2:  12%|█▏        | 3001/25000 [08:58<1:06:33,  5.51it/s]

...


Review: MANNA FROM HEAVEN is a terrific film that is both predictable and unpredictable at the same time. Yo...
True sentiment: positive
Predicted sentiment: unknown
Required 3 attempt(s)


Evaluating GPT-2:  96%|█████████▌| 24001/25000 [1:11:59<03:02,  5.47it/s]


Review: Don't watch this film while, or soon after, eating.<br /><br />Having said that, Begotten will stick...
True sentiment: positive
Predicted sentiment: unknown
Required 3 attempt(s)


Evaluating GPT-2: 100%|██████████| 25000/25000 [1:15:00<00:00,  5.56it/s]


Review: I was expecting "Born to Kill" to be an exciting, high-tension film noir. Instead, it's got two good...
True sentiment: negative
Predicted sentiment: unknown
Required 3 attempt(s)

===== GPT-2 Zero-Shot Sentiment Analysis Results =====
Model: gpt2
Total samples: 25000
Valid predictions: 54 (0.22%)
Unknown predictions: 24946 (99.78%)


It appears that almost all the predictions is unknown. This could be a result of GPT2 being too small, and thus not being good enough to follow instructions well for answer. Let's try an alternative method using finetuning and the classification head for gpt instead